一. 重返泰坦尼克号
对数据的情况的大致说明，每一项的具体意义，参见Kaggle
二. 我们所关心的问题
三. 对数据进行整理
四. 探索数据
五. 得出结论
要表明结论只是暂时的
六. 沟通阶段
七. 评分标准
代码功能性
代码是否正常运行？所有代码均有用且运行时不会产生错误。所给代码足够重现描述的结果。

项目是否适当使用了 NumPy 和 Pandas？项目在适当情况下使用 NumPy 阵列和 Pandas 系列及 DataFrames，而非 Python 列表与字典。并在可能情况下使用矢量化操作和内置函数，而非循环。

项目是否使用良好的编码实践？代码利用函数来避免重复代码。代码包含清晰的注释和变量名称，可读性较高。

分析质量
项目是否清楚地提出了问题？项目清楚地提出了一个或多个问题，然后在分析的其余部分解决了这些问题。

数据整理阶段
是否对数据清理进行了良好的记录？项目记录了为清理数据所做的所有变更，例如合并多个文件，处理丢失的值等。

探索阶段
是否采用多种方式探索数据？项目从多个角度调查所述问题。至少有三个变量使用单变量 (1d) 和多变量 (2d) 探索进行了调查。

是否存在多种相关可视化和统计汇总？项目采用了多种可视化并展示了多项对比和趋势。分析中在进行与数据有关的推理时都会计算相关统计。至少应创建两种图作为探索的部分。

结论阶段
学员是否正确表明了结论只是暂时的？分析结果在展示时明确说明了存在的任何限制。分析并未表明或暗示一个变更完全基于相关性导致另一个变更。

沟通阶段
分析流程是否容易理解？每个分析决策、图和统计汇总均提供了推理说明。

数据是否使用了适当的图和参数进行了可视化？项目的可视化采用适当的方式描述数据，所用的图容易解读。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
titanic_df = pd.read_csv('titanic-data.csv')
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
